## Private market data API  using krakenex3

##### imports:

In [1]:
from pprint import pprint
import krakenex

import decimal
import time



### Get account balance

URL: https://api.kraken.com/0/private/Balance

Result: array of asset names and balance amount

In [3]:
k = krakenex.API()
k.load_key('../kraken.key')
k.conn = krakenex.Connection()
response = k.query_private('Balance')
k.conn.close()
pprint (response)

{'error': [],
 'result': {'XETH': '0.3945722900', 'XXRP': '160.00000000', 'ZEUR': '0.9540'}}


### Get trade balance

URL: https://api.kraken.com/0/private/TradeBalance

Input:

aclass = asset class (optional):<br>
    currency (default)<br>
asset = base asset used to determine balance (default = ZUSD)

Result: array of trade balance info

eb = equivalent balance (combined balance of all currencies)<br>
tb = trade balance (combined balance of all equity currencies)<br>
m = margin amount of open positions<br>
n = unrealized net profit/loss of open positions<br>
c = cost basis of open positions<br>
v = current floating valuation of open positions<br>
e = equity = trade balance + unrealized net profit/loss<br>
mf = free margin = equity - initial margin (maximum margin available to open new positions)<br>
ml = margin level = (equity / initial margin) * 100<br>

Note: Rates used for the floating valuation is the midpoint of the best bid and ask prices

In [6]:
k = krakenex.API()
k.load_key('../kraken.key')
k.conn = krakenex.Connection()
req={'asset':'ZEUR'}
response = k.query_private('TradeBalance', req = req)
k.conn.close()
pprint (response)


{'error': [],
 'result': {'c': '0.0000',
            'e': '97.7978',
            'eb': '125.6474',
            'm': '0.0000',
            'mf': '97.7978',
            'n': '0.0000',
            'tb': '97.7978',
            'v': '0.0000'}}


### Get open orders

URL: https://api.kraken.com/0/private/OpenOrders

Input:

trades = whether or not to include trades in output (optional.  default = false)<br>
userref = restrict results to given user reference id (optional)<br>

Result: array of order info in open array with txid as the key

refid = Referral order transaction id that created this order<br>
userref = user reference id<br>
status = status of order:<br>
    pending = order pending book entry<br>
    open = open order<br>
    closed = closed order<br>
    canceled = order canceled<br>
    expired = order expired<br>
opentm = unix timestamp of when order was placed<br>
starttm = unix timestamp of order start time (or 0 if not set)<br>
expiretm = unix timestamp of order end time (or 0 if not set)<br>
descr = order description info<br>
    pair = asset pair<br>
    type = type of order (buy/sell)<br>
    ordertype = order type (See Add standard order)<br>
    price = primary price<br>
    price2 = secondary price<br>
    leverage = amount of leverage<br>
    order = order description<br>
    close = conditional close order description (if conditional close set)<br>
vol = volume of order (base currency unless viqc set in oflags)<br>
vol_exec = volume executed (base currency unless viqc set in oflags)<br>
cost = total cost (quote currency unless unless viqc set in oflags)<br>
fee = total fee (quote currency)<br>
price = average price (quote currency unless viqc set in oflags)<br>
stopprice = stop price (quote currency, for trailing stops)<br>
limitprice = triggered limit price (quote currency, when limit based order type triggered)<br>
misc = comma delimited list of miscellaneous info<br>
    stopped = triggered by stop price<br>
    touched = triggered by touch price<br>
    liquidated = liquidation<br>
    partial = partial fill<br>
oflags = comma delimited list of order flags<br>
    viqc = volume in quote currency<br>
    fcib = prefer fee in base currency (default if selling)<br>
    fciq = prefer fee in quote currency (default if buying)<br>
    nompp = no market price protection<br>
trades = array of trade ids related to order (if trades info requested and data available)<br>

Note: Unless otherwise stated, costs, fees, prices, and volumes are in the asset pair's scale, not the currency's scale. For example, if the asset pair uses a lot size that has a scale of 8, the volume will use a scale of 8, even if the currency it represents only has a scale of 2. Similarly, if the asset pair's pricing scale is 5, the scale will remain as 5, even if the underlying currency has a scale of 8.

In [11]:
k = krakenex.API()
k.load_key('../kraken.key')
k.conn = krakenex.Connection()
req={'asset':'ZEUR'}
response = k.query_private('OpenOrders', req = req)
k.conn.close()
pprint (response)


{'error': [], 'result': {'open': {}}}


### Get closed orders

URL: https://api.kraken.com/0/private/ClosedOrders

Input:

trades = whether or not to include trades in output (optional.  default = false)
userref = restrict results to given user reference id (optional)
start = starting unix timestamp or order tx id of results (optional.  exclusive)
end = ending unix timestamp or order tx id of results (optional.  inclusive)
ofs = result offset
closetime = which time to use (optional)
    open
    close
    both (default)

Result: array of order info

closed = array of order info.  See Get open orders.  Additional fields:
    closetm = unix timestamp of when order was closed
    reason = additional info on status (if any)
count = amount of available order info matching criteria

Note: Times given by order tx ids are more accurate than unix timestamps. If an order tx id is given for the time, the order's open time is used

In [12]:
# pair = 'XETHZEUR,XETHZUSD'
# pair = 'XETHZEUR'
# req = {'pair':pair}
k.conn = krakenex.Connection()
k.load_key('../kraken.key')
response = k.query_private('ClosedOrders')

k.conn.close()
pprint (response)

{'error': [],
 'result': {'closed': {'OCCM53-TRQIA-TRCYQG': {'closetm': 1503251846.0685,
                                               'cost': '130.00',
                                               'descr': {'leverage': 'none',
                                                         'order': 'buy '
                                                                  '0.51457229 '
                                                                  'ETHEUR @ '
                                                                  'limit '
                                                                  '252.63',
                                                         'ordertype': 'limit',
                                                         'pair': 'ETHEUR',
                                                         'price': '252.63',
                                                         'price2': '0',
                                                         'type': 'buy'},
            

### Query orders info

URL: https://api.kraken.com/0/private/QueryOrders

Input:

trades = whether or not to include trades in output (optional.  default = false)
userref = restrict results to given user reference id (optional)
txid = comma delimited list of transaction ids to query info about (20 maximum)

Result: associative array of orders info

< order_txid> = order info.  See Get open orders/Get closed orders


In [14]:

k.conn = krakenex.Connection()
k.load_key('../kraken.key')
req = {'txid':'OUTNA7-AIOV6-2KK3EH'}
response = k.query_private('QueryOrders', req=req)

k.conn.close()
pprint (response)

{'error': [],
 'result': {'OUTNA7-AIOV6-2KK3EH': {'closetm': 1504975344.4895,
                                    'cost': '28.76320',
                                    'descr': {'leverage': 'none',
                                              'order': 'buy 160.00000000 '
                                                       'XRPEUR @ market',
                                              'ordertype': 'market',
                                              'pair': 'XRPEUR',
                                              'price': '0',
                                              'price2': '0',
                                              'type': 'buy'},
                                    'expiretm': 0,
                                    'fee': '0.07478',
                                    'misc': '',
                                    'oflags': 'fciq',
                                    'opentm': 1504975343.9647,
                                    'price': '0.17977',
         

### Get trades history

URL: https://api.kraken.com/0/private/TradesHistory

Input:

type = type of trade (optional)
    all = all types (default)
    any position = any position (open or closed)
    closed position = positions that have been closed
    closing position = any trade closing all or part of a position
    no position = non-positional trades
trades = whether or not to include trades related to position in output (optional.  default = false)
start = starting unix timestamp or trade tx id of results (optional.  exclusive)
end = ending unix timestamp or trade tx id of results (optional.  inclusive)
ofs = result offset

Result: array of trade info

trades = array of trade info with txid as the key
    ordertxid = order responsible for execution of trade
    pair = asset pair
    time = unix timestamp of trade
    type = type of order (buy/sell)
    ordertype = order type
    price = average price order was executed at (quote currency)
    cost = total cost of order (quote currency)
    fee = total fee (quote currency)
    vol = volume (base currency)
    margin = initial margin (quote currency)
    misc = comma delimited list of miscellaneous info
        closing = trade closes all or part of a position
count = amount of available trades info matching criteria

If the trade opened a position, the follow fields are also present in the trade info:

    posstatus = position status (open/closed)
    cprice = average price of closed portion of position (quote currency)
    ccost = total cost of closed portion of position (quote currency)
    cfee = total fee of closed portion of position (quote currency)
    cvol = total fee of closed portion of position (quote currency)
    cmargin = total margin freed in closed portion of position (quote currency)
    net = net profit/loss of closed portion of position (quote currency, quote currency scale)
    trades = list of closing trades for position (if available)

Note:

    Unless otherwise stated, costs, fees, prices, and volumes are in the asset pair's scale, not the currency's scale.
    Times given by trade tx ids are more accurate than unix timestamps.


In [18]:
req = {'txid':'OUTNA7-AIOV6-2KK3EH'}
k.conn = krakenex.Connection()
response = k.query_private('TradesHistory', req=req)

k.conn.close()
pprint (response)

{'error': [],
 'result': {'count': 2,
            'trades': {'T26HTS-66X6D-OVKEI2': {'cost': '28.76320000',
                                               'fee': '0.07478432',
                                               'margin': '0.00000000',
                                               'misc': '',
                                               'ordertxid': 'OUTNA7-AIOV6-2KK3EH',
                                               'ordertype': 'market',
                                               'pair': 'XXRPZEUR',
                                               'price': '0.17977000',
                                               'time': 1504975344.4893,
                                               'type': 'buy',
                                               'vol': '160.00000000'},
                       'TEG6C6-UMLZJ-UJWW4V': {'cost': '130.00000',
                                               'fee': '0.20800',
                                               'margin': '0.00000'

### Get open positions

URL: https://api.kraken.com/0/private/OpenPositions

Input:

txid = comma delimited list of transaction ids to restrict output to
docalcs = whether or not to include profit/loss calculations (optional.  default = false)

Result: associative array of open position info

<position_txid> = open position info
    ordertxid = order responsible for execution of trade
    pair = asset pair
    time = unix timestamp of trade
    type = type of order used to open position (buy/sell)
    ordertype = order type used to open position
    cost = opening cost of position (quote currency unless viqc set in oflags)
    fee = opening fee of position (quote currency)
    vol = position volume (base currency unless viqc set in oflags)
    vol_closed = position volume closed (base currency unless viqc set in oflags)
    margin = initial margin (quote currency)
    value = current value of remaining position (if docalcs requested.  quote currency)
    net = unrealized profit/loss of remaining position (if docalcs requested.  quote currency, quote currency scale)
    misc = comma delimited list of miscellaneous info
    oflags = comma delimited list of order flags
        viqc = volume in quote currency

Note: Unless otherwise stated, costs, fees, prices, and volumes are in the asset pair's scale, not the currency's scale.


In [15]:
k.conn = krakenex.Connection()
response = k.query_private('QueryTrades')

k.conn.close()
pprint (response)

{'error': [],
 'result': {'XETHZEUR': {'asks': [['239.08000', '31.900', 1505039302],
                                  ['239.09000', '4.312', 1505039298],
                                  ['239.10000', '42.561', 1505039296],
                                  ['239.12000', '44.654', 1505039284],
                                  ['239.13000', '13.560', 1505039225],
                                  ['239.14000', '1.413', 1505039064],
                                  ['239.50000', '0.050', 1505039296],
                                  ['239.55000', '2.000', 1505039268],
                                  ['239.57000', '1.329', 1505039305],
                                  ['239.58000', '42.000', 1505039245],
                                  ['239.60000', '0.013', 1505039066],
                                  ['239.70000', '3.638', 1505038994],
                                  ['239.72000', '2.000', 1505038852],
                                  ['239.98000', '20.000', 1505039305],


### Get recent spread data

URL: https://api.kraken.com/0/public/Spread

Input:

pair = asset pair to get spread data for
since = return spread data since given id (optional.  inclusive)

Result: array of pair name and recent spread data

< pair_name> = pair name
    array of array entries(< time>, < bid>, < ask>) 
last = id to be used as since when polling for new spread data

Note: "since" is inclusive so any returned data with the same time as the previous set should overwrite all of the previous set's entries at that time

In [17]:
pair = 'XETHZEUR'
since = str(1499000000)  # UTC 2017-07-02 12:53:20
req = {'pair':pair, 'since':since}
k.conn = krakenex.Connection()
response = k.query_public('Spread', req=req)

k.conn.close()
pprint (response)

{'error': [],
 'result': {'XETHZEUR': [[1505038213, '239.03000', '239.07000'],
                         [1505038213, '239.03000', '239.08000'],
                         [1505038235, '239.08000', '239.08000'],
                         [1505038237, '239.03000', '239.08000'],
                         [1505038237, '239.01000', '239.08000'],
                         [1505038240, '239.01000', '239.02000'],
                         [1505038247, '239.02000', '239.02000'],
                         [1505038249, '239.01000', '239.02000'],
                         [1505038267, '239.01000', '239.07000'],
                         [1505038273, '239.01000', '239.01000'],
                         [1505038273, '239.01000', '239.07000'],
                         [1505038276, '239.01000', '239.01000'],
                         [1505038279, '239.00000', '239.01000'],
                         [1505038302, '239.01000', '239.01000'],
                         [1505038304, '239.00000', '239.01000'],
           